# XSPEC Tutorial

XSPEC is part of the NASA HEASoft software suit. It is a command driven, interactive, X-ray spectral fitting program with lots of models pre defined.
https://heasarc.gsfc.nasa.gov/docs/xanadu/xspec/manual/index.html

## Starting XSPEC & loading data

1. Start XSPEC using the command `xspec` in the terminal. This opens XSPEC environment where one can provide data files, perform spectrum fitting.
2. Load the data about spectrum using `data <file_name>`. It supports .fits as well as .grp files. If the ARF and RMF files are present in the same directory, XSPEC will automatically load them.

    <div class="alert alert-block alert-info">
    <b>Data types:</b>
    <br> The four main types of data used for spectral fitting are:
    <br> * Source spectrum: The spectrum of the source that we want to fit.
    <br>* Background spectrum: The spectrum of the background that we want fit for accounting the background in the source spectrum. Needs to be loaded using "backgrnd file_name".
    <br>* ARF (Ancillary Response Files) Response matrix: This file indicates the effective area of the telescope as a function of energy.
    <br>* RMF Response matrix (Response Matrix Files): This file links the instrumental channel scale to the energy scale.
    </div>

3. Once the data is loaded and if the ARF and RMF files are present in the same directory, XSPEC will display the data loaded and the number of channels in the data as shown below.

    ```bash
    XSPEC12>data em01_crab_Spe_820_SourceSpec_00001.fits
                                                                                                                                                                                                
    Spectrum #: 1 replaced                                                                                                                                                                       
                                                                                                                                                                                                
    1 spectrum  in use                                                                                                                                                                           
                                                                                                                                                                                                
    Spectral Data File: em01_crab_Spe_820_SourceSpec_00001.fits  Spectrum 1                                                                                                                      
    Net count rate (cts/s) for Spectrum:1  7.319e+02 +/-  2.718e+00                                                                                                                               
    Assigned to Data Group 1 and Plot Group 1                                                                                                                                                   
    Noticed Channels:  1-1024                                                                                                                                                                  
    Telescope: eROSITA Instrument: MERGED  Channel Type: PI                                                                                                                                    
    Exposure Time: 99.09 sec                                                                                                                                                                   
    Using fit statistic: chi                                                                                                                                                                    
    Using Response (RMF) File            em01_crab_Spe_820_RMF_00001.fits for Source 1                                                                                                          
    Using Auxiliary Response (ARF) File  em01_crab_Spe_820_ARF_00001.fits   
    ```

4. One can rebin the data using the command 
    ```bash 
    setplot rebin <min significance> <max bins>
    ```
    This is useful when the data has too many channels and the fitting is taking too long. Alternatively, one can use the heasoft tool `ftgrouppha` to group the data into a specified number of channels. To use this tool, run the command 
    ```bash
    ftgrouppha infile=<input_file> outfile=<output_file_as_.grp> grouptype=opt respfile=<RMF_file> 
    ```

5. To ignore the channels with energy range, use the command 
    ```bash
    ignore <min_energy>-<max_energy>
    ```
    For example, to keep the channels between 0.3-7 keV, use the command 
    ```bash
    ignore **-0.3 7-**
    ```

6. To load multiple data files, use the command 
    ```bash
    data 1:1 <file1> 2:2 <file2> 3:3 <file3> 
    ```
    To ignore a particular energy range in all files, use the command 
    ```bash
    ignore 1-3: **-0.3 7-**
    ```

## Plotting the data
1. First step to plot the data is to initialize the plotting window using the command 
    ```bash
    cpd <device_name>
    ```
    The device name can be `/xw` for X-window, `/ps` for postscript file, `/png` for png file, etc.

2. To plot the data, use the command 
    ```bash
    plot data
    ```
    The command `plot data` plots the data as it is, while `plot ldata` plots the log of the data. This plot consists `channels` on x-axis and `counts` on y-axis.

3. To plot energy [kev] on x-axis and [counts /s /keV] on y-axis, use the command 
    ```bash
    setplot energy
    plot ldata
    ```
    This plot consists `energy` on x-axis and `flux` on y-axis.

4. To plot the background data, use the command 
    ```bash
    setplot background
    plot ldata
    ```
    To rescale y-axis, use the command `setplot command r y` and to remove the background plot, use the command `setplot noback`.

Once this is done, the file should look like this:
<p align="center">
    <img src="Images/2_Energy_BG.png" alt="plot" style="width:40%;">
</p>

## Choosing the model and fitting the data

XSPEC has a large number of predefined models. To see the list of models, use the command `model` or `mo`. There are three types of models: `additive`, `multiplicative`, and `other`. The `additive` models are added to the source spectrum, the `multiplicative` models are multiplied to the source spectrum, and the `other` models are convolution models, Mixin models, and Pile-up models.

1. For example, to use the power-law model, use the command 
    ```bash
    mo TBabs*po
    ```
    Here, `TBabs` is the absorption model and `po` is the power-law model. The model `TBabs` requires the Hydrogen column density ($nH$) as a parameter in the units of $10^{22} cm^{-2}$. 

    <div class="alert alert-block alert-info">
    <b>nH value:</b>
    <br> If the value of nH is unknown, use the command "nh" to find the value for a given RA and DEC.
    </div>

2. When the command `mo TBabs*po` is executed, input the values of the parameters such as `nH`, `phoindex`, `norm`. To freeze a parameter, add -1 at the end of the parameter value. Once the values are input, a summary of the model is displayed.

3. Use the command `query yes` to ensure the fitting continues without any questions being asked. Use the command `renorm` to adjust the normalization of the model to the data which is chosen to minimize the fit statistic.

4. To fit the data, use the command `fit`. One can also add number of iterations to the fit command to get a better fit. For example, `fit 100` will fit the data 100 times. Once the fit is performed, the output shows the best-fit parameters and the fit statistic. The reduced $\chi^2$ value for this fit is more than 2 and thus it is not a good fit.

5. To plot the model along with the residual plot, use the command 
    ```bash
    plot ldata delchi
    ```
    The output will look like this:

    <p align="center">
        <img src="Images/3_Data_Model_delchi.png" alt="plot" style="width:40%;">
    </p>

6. If the fit is not good, one can change the model or the parameters and refit the data. To add a new component to the model, such as a gaussian line, use the command 
    ```bash 
    addcomp 2 gaussian
    ```
    Here 2 refers to adding this gaussian to the second model which was powerlaw. Add the input parameters for the gaussian model and fit the data again. In the previous image, one can see the gaussian line needs to be added at around 0.7 keV and 2.5 kev. Once they are added, the model will be described as `TBabs*(po+gaussian+gaussian)` and the fit looks like this:
    <p align="center">
        <img src="Images/4_Data_Gauss_Model_delchi.png" alt="plot" style="width:40%;">
    </p>

    This has reduce $\chi^2$ value of 1.33 and this can be used to calculate errors on the parameters.

7. To calculate the errors on the parameters, use the command `error 1.0 2`. This will calculate the errors on the parameter 2 with 1 sigma confidence level. 

8. To save the fit results in a file, use the command `save all <file_name>`. This will save the fit results in a file with the name `file_name` in the current directory.

9. To calculate the flux of the source, use the command 
    ```bash
    flux 0.3 7
    ``` 
    This will calculate the flux of the source between 0.3-7 keV.

## Background modeling

To account for the background in the source spectrum, one can used the extracted background spectrum to model the background. Instead of subtracting the background from the source spectrum, one can model the background and fit the source spectrum along with the background model. 

Background here consists of two components: astrophysical background and instrumental background. The astrophysical background consists of the following components:
1. Local Hot Bubble (LHB) which is a low temperature plasma in the solar neighborhood and is represented by a collisionally ionized diffuse gas model called `apec`.
2. Galactic Halo (GH) which is a hot plasma in the Galactic halo and is represented by an absorbed `apec` model (`TBabs*apec`).
3. Cosmic X-ray Background (CXB) which is the unresolved X-ray emission from all extragalactic sources such as AGNs and is represented by an absorbed power-law model (`TBabs*po`).
This gives the model for the astrophysical background as `TBabs*(apec+po)+apec`.

The instrumental background consists of the following components:
1. Particle induced background (PIB) which is the background due to the particles hitting the detector and is represented by a power-law model.
2. Fluorescent lines which are the lines due to the fluorescence of the detector material and are represented by a gaussian model.
The model for the instrumental background is obtained from the Filter Wheel Closed (FWC) data and is available on the eROSITA website. The model for the instrumental background is represented as a combination of 23 gaussian lines, one power-law model, and `expfac(bkn2pow + powerlaw + powerlaw)`

To model the background, start with loading the background spectrum as a source spectrum using the command `data <background_file>`.

1. Use the following command to model the astrophysical background. 
    ```bash
    mo TBabs*(apec+po)+apec
    ```
2. Save the model using the command `save all <file_name.xcm>`. This will save the model as well as the information about the input file in the file `file_name.xcm` in the current directory. The model will look like this:
    ```bash
    model  TBabs(apec + powerlaw) + apec 
       0.409637      0.001          0          0     100000      1e+06
       0.296085       0.01      0.008      0.008         64         64
       ...
    ```

3. Copy and paste the model from the FWC data file into this file such that your model looks like this:
    ```bash
    model TBabs(apec + powerlaw) + apec + gaussian + gaussian + gaussian + gaussian + gaussian + gaussian + gaussian + gaussian + gaussian + gaussian + gaussian + gaussian + gaussian + gaussian + expfac(bkn2pow + powerlaw + powerlaw) + gaussian + powerlaw + gaussian + gaussian + gaussian + gaussian + gaussian + gaussian + gaussian + gaussian
       0.409637      0.001          0          0     100000      1e+06
       0.296085       0.01      0.008      0.008         64         64
       ...
        9.49841       0.01       9.45        9.5        9.7        9.8
         0.0001      -0.05          0          0         10         20
      0.0018608       0.01          0          0      1e+20      1e+24
       ...
    ```

4. Load the model using the command `@<file_name.xcm>`. This will load the model into the XSPEC environment.

5. Perform the fit using the command `fit 100` if necessary. Once the fit is done, the output will show the best-fit parameters and the reduced $\chi^2$ value. 

6. To plot the model along with its components use the command 
    ```bash
    setplot energy
    setplot add
    plot ldata 
    iplot
    PLT> rescale y 1e-4 100
    PLT> col 2 on 2
    PLT> plot
    ```
    The later commands are used to rescale the y-axis and change the color of the final model to red.

The output will look like this:
    <p align="center">
        <img src="Images/5_BG_Modeling.png" alt="plot" style="width:40%;">
    </p>

Use this background model and its parameters when modeling source spectrum. 
For ease of use, this part has been done using the python package `PyXspec` which is a python implementation of XSPEC. PyXspec tutorial on how to fit and model the data can be found at [PyXSPEC.ipynb](PyXSPEC.ipynb).

<div class="alert alert-block alert-danger">
<b>Important note:</b> This tutorial is a brief introduction to XSPEC and its functionalities. It is prepared to understand basics of the software. The methods described here are not exhaustive and definitely requires better implementation for a more accurate and precise analysis.
</div>